In [ ]:
import torch
import os
import torchaudio
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
DATA_PATH = '../../data/music_genre_dataset'

WAV_PATH = os.path.join(DATA_PATH, 'genres_original')
TEST_PATH = os.path.join(DATA_PATH, 'test_set')

In [ ]:
### Arguments
### path: path of either wav or png directory
### Returns
### dictionary of 'genre' -> number
def extract_classes(path=WAV_PATH):
    classes = {}
    i = 0
    for genre in os.listdir(path):
        classes[genre] = i
        i += 1
    return classes
CLASSES = extract_classes()
print(CLASSES)

The classification of genre is enumerated in the following way: {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [ ]:
def get_genre_lable_vector(genre):
    if genre == 'blues':
        return torch.tensor([[1,0,0,0,0,0,0,0,0,0]])
    elif genre == 'classical':
        return torch.tensor([[0,1,0,0,0,0,0,0,0,0]])
    elif genre == 'country':
        return torch.tensor([[0,0,1,0,0,0,0,0,0,0]])
    elif genre == 'disco':
        return torch.tensor([[0,0,0,1,0,0,0,0,0,0]])
    elif genre == 'hiphop':
        return torch.tensor([[0,0,0,0,1,0,0,0,0,0]])
    elif genre == 'jazz':
        return torch.tensor([[0,0,0,0,0,1,0,0,0,0]])
    elif genre == 'metal':
        return torch.tensor([[0,0,0,0,0,0,1,0,0,0]])
    elif genre == 'pop':
        return torch.tensor([[0,0,0,0,0,0,0,1,0,0]])
    elif genre == 'reggae':
        return torch.tensor([[0,0,0,0,0,0,0,0,1,0]])
    else:
        return torch.tensor([[0,0,0,0,0,0,0,0,0,1]])

In [ ]:
'''
Returns a set of tensors with vectorized wav files and the labels for each wav file.

train: tensor([[0.3,0.2,...,n],[0.1,0.4,...,n],...,z]),
label: tensor(z,10)
}
'''
def getTrainDataSet(path, duration=10, sr=22050):
    genres = os.listdir(path)
    n_frame = duration * sr        
    audio_tensor = torch.Tensor()
    label_tensor = torch.Tensor()
    for genre in genres:
        genre_path = os.path.join(path, genre)
        files = os.listdir(genre_path)
        for file in files:
            file_path = os.path.join(genre_path, file)
            audio_frames = torchaudio.load(file_path, num_frames=n_frame)
            audio_tensor = torch.cat((audio_tensor, audio_frames[0]), dim=0)            
            label_tensor = torch.cat((label_tensor, get_genre_lable_vector(genre)), dim=0)
    return audio_tensor, label_tensor

train, label = getTrainDataSet(WAV_PATH)
print(train.shape)
print(label.shape)
print(train)
print(label)

In [ ]:
test, test_label = getTrainDataSet(TEST_PATH)

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_tensor = TensorDataset(train, label)
test_tensor = TensorDataset(test, test_label)

train_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
input_layer_size = 0

for X, y in train_loader:
    input_layer_size = X.shape[1]
    print(X.shape[1])
    break

In [ ]:
torch.full(size=(10,), fill_value=1)

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_layer_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax()          
        )

    def forward(self, x):        
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [ ]:
# Initialize the loss function
loss_fn = nn.MSELoss() # mean absolute error , mse

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)        
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # fit
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()            
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]

model.eval()
x, y = test_tensor[0][0], test_tensor[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')